# Boundary Conditions

Checking if boundary conditions are properly added.

In [1]:
%matplotlib inline

from dolfin import *
import math as m
import numpy as np
import matplotlib 
import matplotlib.pyplot as plt

nx = 20
P = 1

mesh = UnitSquareMesh(nx,nx)
h = CellDiameter(mesh)
Q = FunctionSpace(mesh, "CG", P)

In [3]:
from __future__ import print_function
import sympy as sym
x, y, t = sym.symbols('x[0], x[1], t')
sigma = 1.0
mu = 10**(-6)
velocity = as_vector([1.0, 1.0])
a = velocity[0]
b = velocity[1]

ue = x + y + t**2

# Exact Solution
ue = sym.simplify(ue)
u_code = sym.printing.ccode(ue)
u_code = u_code.replace('M_PI','DOLFIN_PI')

print(u_code)

# du/dt - mu*Laplace(u) + div(bu) + sigma*u = f
fe = sym.diff(ue,t) - mu*(sym.diff(sym.diff(ue,x),x) + sym.diff(sym.diff(ue,y),y))
fe += a*sym.diff(ue,x) + b*sym.diff(ue,y)
fe += sigma*ue

f_code = sym.printing.ccode(fe)
f_code = f_code.replace('M_PI','DOLFIN_PI')
print(f_code)


pow(t, 2) + x[0] + x[1]
1.0*pow(t, 2) + 2*t + 1.0*x[0] + 1.0*x[1] + 2.0


In [4]:
f = Expression(f_code, degree = P, t=0.0)
u_n = Function(Q)
u_D = Constant(0.0)
dt = 0.001
S = 1.0
T = 1.0

# Set up boundary condition
def boundary(x, on_boundary):
    return on_boundary
bc = DirichletBC(Q, u_D, boundary)


# Don't Modify Below This! -----------#

# Test and trial functions
u, v = TrialFunction(Q), TestFunction(Q)
u_ = Function(Q)
u_bar = Function(Q)

# Galerkin variational problem
# Backward Euler
F = v*(u - u_n)*dx + dt*(mu*dot(grad(v), grad(u))*dx \
                        + v*dot(velocity, grad(u))*dx \
                        + sigma*v*u*dx \
                        - f*v*dx)

# --- Begin EFR --- #
delta = S*1.0/nx

u_tilde0 = Function(Q)


# Define indicator function to evaluate current time step
def a(u_tilde, u_, t):
    indicator = Expression('sqrt((a-b)*(a-b))', degree = 2, a = u_, b = u_tilde)
    indicator = interpolate(indicator, Q)
    max_ind = np.amax(indicator.vector().get_local())#.vector().array())

    # Normalize indicator such that it's between [0,1].
    if max_ind < 1:
       max_ind = 1.0

    indicator = Expression('a/b', degree = 2, a = indicator, b = max_ind)
    indicator = interpolate(indicator, Q) 
    indicator.rename('a','a')
    #out_file_ind << (indicator, float(t))
    return indicator


--- Instant: compiling ---


In [27]:
# Define variational problem for step 2a (apply Helmholz filter)
# Note: only RHS changes, so we can keep the same a2 throughout

a2 = v*u*dx + delta*delta*dot(grad(v), grad(u))*dx #lhs(F_Hfilter)
A2 = assemble(a2)

def L2(u_): # input is intermediate velocity OR previous u_tilde solution
    L2 = v*u_*dx
    return L2

# Define variational problem for step 2b (evaluate indicator and find filtered solution)
def a3(ind):
    a3 = v*u*dx + delta*delta*dot(grad(v), ind*grad(u))*dx
    return a3

L3 = v*u_*dx

# --- End EFR --- #

num_steps = int(round(T / dt, 0)) 

# Create bilinear and linear forms
a1 = lhs(F)
L = rhs(F)

Calling FFC just-in-time (JIT) compiler, this may take some time.


In [30]:
# Assemble matrices
A1 = assemble(a1)
bc.apply(A1)

<Matrix wrapper of <PETScMatrix of size 441 x 441>>


In [ ]:
# Step 1 Solve on Coarse Grid
#b = assemble(L)
#bc.apply(b)
#solve(A1, u_.vector(), b, "gmres")

# Step 2a Solve Helmholtz filter
# N=0
b2_0 = assemble(L2(u_))
bc_0 = DirichletBC(Q, u_, boundary)
bc_0.apply(b2_0)
bc_0.apply(A2)
solve(A2, u_tilde0.vector(), b2_0, "cg")
DF = Expression('a', degree = R, a=u_tilde0)

# Step 2b Calculate Indicator and solve Ind Problem
ind = a(DF, u_, float(t))

A3 = assemble(a3(ind))
bc.apply(A3)

b3 = assemble(L3)
bc.apply(b3)

solve(A3, u_bar.vector(), b3, "gmres")

progress.update(t / T)

# Update previous solution and source term
u_n.assign(u_bar)
# Update current time
t += dt
f.t += dt